In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

 
DATA_DIR = './data/'
PREDICTION_DIR = './prediction/'
LATENT_DIM = 128  # Dimesion of the compressed representation
EPOCHS_AE = 50      # Epochs for Autoencoder training
EPOCHS_PRED = 100   # Epochs for Predictor training
BATCH_SIZE = 64
LEARNING_RATE = 1.5e-4

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [2]:
## 2. Data Loading and Preprocessing (Corrected)

print("Loading and preprocessing data...")

# Load the training data
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_set.csv'), index_col=0)

# Transpose the dataframe so experiments are rows and genes are columns
train_df = train_df.T

# Separate single and double perturbations
single_pert_mask = train_df.index.str.contains('\+ctrl')
double_pert_mask = ~single_pert_mask

single_gene_df = train_df[single_pert_mask]
double_gene_df = train_df[double_pert_mask]

# Create a clean dictionary for single-gene profiles
single_gene_df.index = single_gene_df.index.str.replace('\+ctrl', '', regex=True)
single_gene_profiles = single_gene_df.T.to_dict('list')

# Convert all data to numpy arrays for easier handling
gene_ids = train_df.columns.tolist()
X_single_all = single_gene_df.values.astype(np.float32)

# --- START: NEW FILTERING LOGIC ---
print(f"Original double-gene pairs: {len(double_gene_df)}")
# Check which double-perturbation pairs have both corresponding single profiles
pert_pairs_train = double_gene_df.index.str.split('+', expand=True)
gene_A_keys = pert_pairs_train.get_level_values(0)
gene_B_keys = pert_pairs_train.get_level_values(1)

valid_mask = [(gA in single_gene_profiles and gB in single_gene_profiles) for gA, gB in zip(gene_A_keys, gene_B_keys)]

# Apply the mask to keep only the valid pairs
double_gene_df = double_gene_df[valid_mask]
print(f"Filtered double-gene pairs with complete data: {len(double_gene_df)}")
# --- END: NEW FILTERING LOGIC ---


# Prepare the training data for the Predictor model using the filtered dataframe
pert_pairs_train = double_gene_df.index.str.split('+', expand=True)
gene_A_train = pert_pairs_train.get_level_values(0)
gene_B_train = pert_pairs_train.get_level_values(1)

# This will now succeed because we've removed the problematic pairs
X_A_train = np.array([single_gene_profiles[g] for g in gene_A_train]).astype(np.float32)
X_B_train = np.array([single_gene_profiles[g] for g in gene_B_train]).astype(np.float32)

# The target variable is the double-gene profile
y_predictor_train = double_gene_df.values.astype(np.float32)

print(f"Found {len(single_gene_df)} single-gene profiles.")
print(f"Using {len(double_gene_df)} double-gene profiles to train the predictor.")

Loading and preprocessing data...
Original double-gene pairs: 3040
Filtered double-gene pairs with complete data: 3000
Found 4040 single-gene profiles.
Using 3000 double-gene profiles to train the predictor.


In [3]:
# --- 3.1 Define the Autoencoder Model ---
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# --- 3.2 Train the Autoencoder ---
print("\nTraining Autoencoder...")
input_dim = X_single_all.shape[1]
autoencoder = Autoencoder(input_dim, LATENT_DIM).to(device)
criterion_ae = nn.MSELoss()
optimizer_ae = optim.Adam(autoencoder.parameters(), lr=LEARNING_RATE)

# Create DataLoader
ae_dataset = TensorDataset(torch.from_numpy(X_single_all))
ae_loader = DataLoader(ae_dataset, batch_size=BATCH_SIZE, shuffle=True)

for epoch in range(EPOCHS_AE):
    total_loss = 0
    for data in ae_loader:
        inputs = data[0].to(device)
        optimizer_ae.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion_ae(outputs, inputs)
        loss.backward()
        optimizer_ae.step()
        total_loss += loss.item()
    print(f'AE Epoch [{epoch+1}/{EPOCHS_AE}], Loss: {total_loss/len(ae_loader):.6f}')

# Separate encoder and decoder for later use
encoder = autoencoder.encoder
decoder = autoencoder.decoder


Training Autoencoder...
AE Epoch [1/50], Loss: 0.675393
AE Epoch [2/50], Loss: 0.153221
AE Epoch [3/50], Loss: 0.151647
AE Epoch [4/50], Loss: 0.147718
AE Epoch [5/50], Loss: 0.142288
AE Epoch [6/50], Loss: 0.140653
AE Epoch [7/50], Loss: 0.139299
AE Epoch [8/50], Loss: 0.136445
AE Epoch [9/50], Loss: 0.133892
AE Epoch [10/50], Loss: 0.132756
AE Epoch [11/50], Loss: 0.131555
AE Epoch [12/50], Loss: 0.130543
AE Epoch [13/50], Loss: 0.129759
AE Epoch [14/50], Loss: 0.128376
AE Epoch [15/50], Loss: 0.127328
AE Epoch [16/50], Loss: 0.126570
AE Epoch [17/50], Loss: 0.126566
AE Epoch [18/50], Loss: 0.125336
AE Epoch [19/50], Loss: 0.124883
AE Epoch [20/50], Loss: 0.124714
AE Epoch [21/50], Loss: 0.123711
AE Epoch [22/50], Loss: 0.122525
AE Epoch [23/50], Loss: 0.122275
AE Epoch [24/50], Loss: 0.121351
AE Epoch [25/50], Loss: 0.120634
AE Epoch [26/50], Loss: 0.120473
AE Epoch [27/50], Loss: 0.119977
AE Epoch [28/50], Loss: 0.118608
AE Epoch [29/50], Loss: 0.117781
AE Epoch [30/50], Loss: 0.1

In [4]:
# --- 4.1 Define the Predictor Model ---
class Predictor(nn.Module):
    def __init__(self, latent_dim):
        super(Predictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim * 2, 256), # Takes concatenated latent vectors
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim) # Outputs a predicted latent vector
        )

    def forward(self, x):
        return self.model(x)

# --- 4.2 Transform data to latent space and train ---
print("\nTraining Predictor Model...")

# Transform all data to latent space using the trained encoder
with torch.no_grad():
    encoder.eval()
    X_A_latent = encoder(torch.from_numpy(X_A_train).to(device)).cpu().numpy()
    X_B_latent = encoder(torch.from_numpy(X_B_train).to(device)).cpu().numpy()
    y_predictor_latent = encoder(torch.from_numpy(y_predictor_train).to(device)).cpu().numpy()

# Concatenate the latent vectors of gene A and B
X_predictor_latent = np.concatenate([X_A_latent, X_B_latent], axis=1)

# Create DataLoader for the predictor
pred_dataset = TensorDataset(torch.from_numpy(X_predictor_latent), torch.from_numpy(y_predictor_latent))
pred_loader = DataLoader(pred_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize and train the predictor
predictor = Predictor(LATENT_DIM).to(device)
criterion_pred = nn.MSELoss()
optimizer_pred = optim.Adam(predictor.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS_PRED):
    total_loss = 0
    for data in pred_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer_pred.zero_grad()
        outputs = predictor(inputs)
        loss = criterion_pred(outputs, targets)
        loss.backward()
        optimizer_pred.step()
        total_loss += loss.item()
    print(f'Predictor Epoch [{epoch+1}/{EPOCHS_PRED}], Loss: {total_loss/len(pred_loader):.6f}')


Training Predictor Model...
Predictor Epoch [1/100], Loss: 2.073958
Predictor Epoch [2/100], Loss: 0.514933
Predictor Epoch [3/100], Loss: 0.406230
Predictor Epoch [4/100], Loss: 0.389918
Predictor Epoch [5/100], Loss: 0.376492
Predictor Epoch [6/100], Loss: 0.365632
Predictor Epoch [7/100], Loss: 0.356757
Predictor Epoch [8/100], Loss: 0.349047
Predictor Epoch [9/100], Loss: 0.342167
Predictor Epoch [10/100], Loss: 0.336875
Predictor Epoch [11/100], Loss: 0.334087
Predictor Epoch [12/100], Loss: 0.330117
Predictor Epoch [13/100], Loss: 0.327400
Predictor Epoch [14/100], Loss: 0.324605
Predictor Epoch [15/100], Loss: 0.324005
Predictor Epoch [16/100], Loss: 0.320371
Predictor Epoch [17/100], Loss: 0.316862
Predictor Epoch [18/100], Loss: 0.316506
Predictor Epoch [19/100], Loss: 0.315259
Predictor Epoch [20/100], Loss: 0.312726
Predictor Epoch [21/100], Loss: 0.312280
Predictor Epoch [22/100], Loss: 0.310915
Predictor Epoch [23/100], Loss: 0.309377
Predictor Epoch [24/100], Loss: 0.307

In [5]:
print("\nGenerating predictions for the test set...")

# Load test set gene pairs
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test_set.csv'), header=None)
test_df.columns = ['perturbation']
pert_pairs_test = test_df['perturbation'].str.split('+', expand=True)
gene_A_test = pert_pairs_test[0]
gene_B_test = pert_pairs_test[1]

# Get single-gene profiles for test pairs
X_A_test = np.array([single_gene_profiles[g] for g in gene_A_test]).astype(np.float32)
X_B_test = np.array([single_gene_profiles[g] for g in gene_B_test]).astype(np.float32)

# Full prediction pipeline
encoder.eval()
predictor.eval()
decoder.eval()
final_predictions = []

with torch.no_grad():
    # 1. Encode single profiles
    latent_A = encoder(torch.from_numpy(X_A_test).to(device))
    latent_B = encoder(torch.from_numpy(X_B_test).to(device))

    # 2. Predict combined latent profile
    latent_AB_input = torch.cat([latent_A, latent_B], dim=1)
    predicted_latent_AB = predictor(latent_AB_input)

    # 3. Decode back to full gene expression space
    predicted_expression_AB = decoder(predicted_latent_AB).cpu().numpy()

# --- Format for submission ---
print("Formatting submission file...")
submission_list = []
for i, pert_name in enumerate(test_df['perturbation']):
    for j, gene_id in enumerate(gene_ids):
        submission_list.append({
            'gene': gene_id,
            'perturbation': pert_name,
            'expression': predicted_expression_AB[i, j]
        })

submission_df = pd.DataFrame(submission_list)

# Save to CSV
if not os.path.exists(PREDICTION_DIR):
    os.makedirs(PREDICTION_DIR)
submission_path = os.path.join(PREDICTION_DIR, 'prediction.csv')
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ Successfully created submission file at: {submission_path}")


Generating predictions for the test set...
Formatting submission file...

✅ Successfully created submission file at: ./prediction/prediction.csv


In [6]:
#0.17627